In [49]:
from collections import defaultdict
import numpy as np
from scipy.sparse import csr_matrix

def read_graph(file_path):
    """
    Reads a graph from a file and returns a dictionary representing the adjacency list.
    Each node and its outgoing links are parsed.
    """
    graph = defaultdict(list)
    max_node = -1  # Start with -1 to ensure even an empty file results in a correct n of 0.
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split(':')
            node = int(parts[0])
            max_node = max(max_node, node)  # Update max_node with the highest node number seen so far.
            if len(parts) > 1 and parts[1]:
                linked_nodes = list(map(int, parts[1].split(',')))
                max_node = max(max_node, *linked_nodes)  # Also check linked nodes to ensure all are counted.
            else:
                linked_nodes = []
            graph[node] = linked_nodes
    print(graph)
    return graph, max_node + 1  # max_node + 1 because node indices start at 0

def create_sparse_matrix(graph, n):
    """
    Converts a graph dictionary to a sparse matrix representation.
    Here, rows are target nodes and columns are source nodes.
    """
    row_indices = []
    col_indices = []
    data = []

    for source_node, target_nodes in graph.items():
        if target_nodes:
            weight = 1 / len(target_nodes)
            for target_node in target_nodes:
                row_indices.append(target_node)
                col_indices.append(source_node)
                data.append(weight)
        else:
            # Here, no explicit handling required, as these nodes should contribute to the teleport probability uniformly.
            continue

    # Create the CSR matrix
    matrix = csr_matrix((data, (row_indices, col_indices)), shape=(n, n))
    return matrix

# Example usage
file_path = 'webgraph.txt'
graph, nodes = read_graph(file_path)
sparse_matrix = create_sparse_matrix(graph, nodes)

print(sparse_matrix.toarray())  # Convert to dense array to view


defaultdict(<class 'list'>, {0: [1, 2, 3], 1: [2, 4], 2: [5], 3: [6, 7], 4: [5, 6], 5: [8], 6: [9], 7: [8, 9], 8: [9], 9: [9]})
[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.33333333 0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.33333333 0.5        0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.33333333 0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.5        0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         1.         0.         0.5        0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.5        0.5        0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.5        0.         0.
  0.         0.         0.         0.        ]
 [0.         0. 

In [52]:
import numpy as np
from scipy.sparse import csr_matrix

def pagerank(matrix, n, damping=0.85, tol=1e-3):
    """
    Computes the PageRank of each node using the power iteration method until convergence.

    Parameters:
        matrix (csr_matrix): The sparse matrix representation of the graph.
        n (int): Number of nodes.
        damping (float): Damping factor for PageRank.
        tol (float): Tolerance for convergence.
    """
    # Initialize the rank vector with equal probabilities
    rank = np.ones(n) / n
    print(f"Initial rank distribution: {rank}")

    # Teleportation vector (handles dead ends)
    teleport = np.ones(n) / n * (1 - damping)
    print(f"Teleportation vector: {teleport}")

    count = 0
    while True:
        count += 1
        new_rank = damping * matrix.dot(rank) + teleport
        # Check convergence
        if np.linalg.norm(new_rank - rank, 1) <= tol:
            print(f"Converged after {count} iterations.")
            break
        rank = new_rank

    print(f"Final PageRank distribution: {rank}")
    write_pagerank_to_file(rank , )
    return rank


defaultdict(<class 'list'>, {0: [1, 2, 3], 1: [2, 4], 2: [5], 3: [6, 7], 4: [5, 6], 5: [8], 6: [9], 7: [8, 9], 8: [9], 9: [9]})
Initial rank distribution: [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
Teleportation vector: [0.015 0.015 0.015 0.015 0.015 0.015 0.015 0.015 0.015 0.015]
Converged after 6 iterations.
Final PageRank distribution: [0.015      0.01925    0.02743125 0.01925    0.02318125 0.04816859
 0.03303328 0.02318125 0.06579534 0.72570904]
PageRank of each node:
[0.015      0.01925    0.02743125 0.01925    0.02318125 0.04816859
 0.03303328 0.02318125 0.06579534 0.72570904]
Sum of PageRank values (should be 1.0): 1.0000000000000002


In [ ]:
def write_pagerank_to_file(ranks, output_file):
    """
    Writes the PageRank values to a specified file in scientific notation with 10 decimal places.

    Parameters:
        ranks (list or numpy array): The PageRank values to be written.
        output_file (str): The path to the file where the PageRank values will be written.
    """
    with open(output_file, "w") as file:
        for rank in ranks:
            file.write(f"{rank:.10e}\n")

    print(f"PageRank values have been successfully written to {output_file}")


In [56]:
def main(input_file):
    graph,nodes = read_graph(input_file)
    sparse_matrix = create_sparse_matrix(graph,nodes)
    ranks = pagerank(sparse_matrix, nodes)

    # Print the PageRank vector to stdout in scientific notation
    for rank in ranks:
        print(f"{rank:.10e}")

if __name__ == "__main__":
    # if len(sys.argv) != 3:
    #     print("Usage: python PageRank.py input.txt d")
    # else:
        # input_file = sys.argv[1]
        # damping_factor = sys.argv[2]
        main('webgraph.txt')

Streaming output truncated to the last 5000 lines.
8.8354749378e-05
2.0064616364e-04
1.1856983627e-04
5.9489274265e-05
1.5845852987e-04
7.0533348696e-05
9.8472777597e-05
7.7381372999e-05
8.3112149353e-05
1.1906719156e-04
8.2804017472e-05
8.4046520956e-05
1.2205083993e-04
9.7848026398e-05
1.1501880886e-04
8.3413238959e-05
1.3951960254e-04
1.0539752227e-04
8.5191621197e-05
1.5822813629e-04
1.2221764398e-04
8.1835428597e-05
7.2924072439e-05
9.3557358069e-05
1.1695269250e-04
1.0440335166e-04
9.1455471161e-05
9.0564844808e-05
1.0454248372e-04
7.9240103713e-05
1.0690840454e-04
8.9449046500e-05
1.0027641415e-04
6.8728898792e-05
8.9217339763e-05
9.9750415158e-05
1.1138784227e-04
1.2042822553e-04
1.0713228717e-04
1.2523832954e-04
1.3025030913e-04
1.1547620062e-04
9.7987309288e-05
7.7329059655e-05
1.0875786846e-04
1.3950574225e-04
8.6381605655e-05
8.4423550476e-05
8.0513763340e-05
1.4091581083e-04
1.0882227185e-04
1.0960574154e-04
1.2111510845e-04
1.1414143793e-04
1.0114184826e-04
7.3912169647e-

In [55]:
import numpy as np
import random

def generate_web_graph(num_pages, max_links=10):
    """
    Generates a web graph where each page links to several other pages randomly.

    Parameters:
        num_pages (int): The number of webpages in the graph.
        max_links (int): Maximum number of outgoing links from any webpage.

    Returns:
        dict: A dictionary representing the web graph. Each key is a node, and the value is a list of nodes it points to.
    """
    graph = {}
    for page in range(num_pages):
        # Determine the number of outgoing links for this node (at least 1 to avoid dangling nodes)
        num_links = random.randint(1, max_links)
        # Ensure no self-links and avoid duplicate links
        links = set()
        while len(links) < num_links:
            link = random.randint(0, num_pages - 1)
            if link != page:
                links.add(link)
        graph[page] = list(links)

    return graph

def write_graph_to_file(graph, filename="webgraph.txt"):
    """
    Writes the generated web graph to a file.

    Parameters:
        graph (dict): The web graph as a dictionary.
        filename (str): The filename to write the graph to.
    """
    with open(filename, 'w') as file:
        for node, links in graph.items():
            links_str = ','.join(map(str, links))
            file.write(f"{node}:{links_str}\n")

# Parameters
num_pages = 10000  # Total number of web pages
max_links = 50     # Maximum number of outgoing links from any webpage

# Generate the graph
web_graph = generate_web_graph(num_pages, max_links)

# Write the graph to a file
write_graph_to_file(web_graph)
